In [22]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mesa.space import MultiGrid
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

In [23]:
class FinancialAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.number_of_shares = 1
        self.opinion_vol= self.model.sigma + random.uniform(0, 0.1) #sigma is interactive parameter
        self.news_sensitivity= random.uniform(0, self.model.maximum_news_sensitivity) #maximum_news_sensitivity is also an interactive parameter
        self.base_propensity_to_sentiment_contagion= random.uniform(0, self.model.maximum_base_propensity_to_sentiment_contagion)
        # maximum_base_propensity_to_sentiment_contagion is also an interactive parameter
        self.propensity_to_sentiment_contagion = self.base_propensity_to_sentiment_contagion
        self.my_sentiment=0
        
    def step(self):
        #updating market sentiment
        if (self.model.returns> 0):
            if (news_arrival(FinancialMarket) > 0):
                self.propensity_to_sentiment_contagion += self.model.returns
            elif (news_arrival(FinancialMarket) < 0):
                self.propensity_to_sentiment_contagion -= self.model.returns
        elif (self.model.returns < 0):
            if (news_arrival(FinancialMarket) < 0):
                self.propensity_to_sentiment_contagion -= self.model.returns
            elif (news_arrival(FinancialMarket) > 0):
                self.propensity_to_sentiment_contagion += self.model.returns
            
        sentiment= [agent.my_sentiment for agent in self.model.grid.get_neighbors(self.pos,moore=True,include_center=False,radius=1)]
        if((self.propensity_to_sentiment_contagion* sum (sentiment)+ self.news_sensitivity *news_arrival(FinancialMarket) + np.random.normal(self.model.miu,self.opinion_vol))> 0):
          #miu is also an interactive parameter
            self.my_sentiment= 1
            self.number_of_shares += 1 #Buy
        else:
            self.my_sentiment = -1
            self.number_of_shares -=  1 #Sell
            
       
       
                       

In [24]:
#Write code to extract DJIA Index and VIX for that batch run here
def news_arrival(model):
    if(np.random.normal(0,1) > 0):
         news_qualitative_meaning =1 
    else: 
        news_qualitative_meaning= -1
    return news_qualitative_meaning

In [25]:
def returns(model):
    number_of_traders= model.num_agents
    returns=sum([agent.my_sentiment for agent in model.schedule.agents]) / number_of_traders
    return returns

In [26]:
def logprice(model):
    try:
        log_price =model.log_price+ model.returns
    except:
         log_price= 0 + model.returns
    return log_price

In [27]:
class FinancialMarket(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height,sigma,maximum_news_sensitivity,maximum_base_propensity_to_sentiment_contagion,miu):
        self.running = True
        self.num_agents = N
        self.sigma=sigma
        self.maximum_news_sensitivity= maximum_news_sensitivity
        self.maximum_base_propensity_to_sentiment_contagion= maximum_base_propensity_to_sentiment_contagion
        self.miu=miu
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.returns=returns(self)
        # Create agents
        for i in range(self.num_agents):
            a = FinancialAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
        self.datacollector = DataCollector(
            #model_reporters={"News-Arrival": news_arrival,"Market-Clearing":market_clearing}
            model_reporters={"News-Arrival": news_arrival,"Returns":returns,"logprice":logprice}
            #agent_reporters ={"Wealth":"my_sentiment"}
        )
        
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        a=self.datacollector.get_model_vars_dataframe()
        self.returns=a.iloc[-1,1]
        self.log_price=a.iloc[-1,2]        
        
        

In [28]:
#Run the model a certain number of times: in this case 20 times
model = FinancialMarket(50, 10, 10,0.4,0.7,0.6,0)
for i in range(20):
    model.step()

In [18]:
output = model.datacollector.get_model_vars_dataframe()
output

,News-Arrival,Returns,logprice
0,1,0.28,0.00
1,1,0.36,0.28
2,1,0.40,0.64
3,-1,0.84,1.04
4,1,0.44,1.88
5,-1,0.52,2.32
6,-1,0.52,2.84
7,1,0.56,3.36
8,1,0.32,3.92
9,-1,0.52,4.24


In [9]:
# server.py
#Plot agents in their grid and show their movement
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import ChartModule
from mesa.visualization.UserParam import UserSettableParameter



n_slider1 = UserSettableParameter('slider', "Number of Agents", 100, 2, 20000, 1)
n_slider2 = UserSettableParameter('slider', "sigma", 0.4, 0, 1, 0.001)
n_slider3 = UserSettableParameter('slider', "Maximum News Sensitivity", 0.7, 0, 1, .01)
n_slider4 = UserSettableParameter('slider', "Maximum Propensity to Contagion", 0.6, 0, 1, .01)
n_slider5 = UserSettableParameter('slider', "Bias", 0, -1, 1, .01)




chart1 = ChartModule([{"Label": "Returns",
                      "Color": "Black"}],
                    data_collector_name='datacollector')
chart2= ChartModule([{"Label": "logprice",
                      "Color": "Grey"}],data_collector_name='datacollector')

def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    if agent.my_sentiment > 0:
        portrayal["Color"] = "green"
    else:
        portrayal["Color"] = "red"
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(FinancialMarket,
                       [grid,chart1,chart2],
                       "Simulated Financial Market",
                       {"N": n_slider1, "width": 10, "height": 10,"sigma":n_slider2,"maximum_news_sensitivity":n_slider3,"maximum_base_propensity_to_sentiment_contagion":n_slider4,"miu":n_slider5})

In [10]:
# run.py
#from server import server
server.port = 8535 # The default
server.launch()

Interface starting at http://127.0.0.1:8535
Socket opened!
{"type":"get_params"}
{"type":"reset"}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.69}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.68}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.67}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.66}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.65}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.64}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.63}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.62}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.61}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.6}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.59}
{"type":"submit_params","param":"maximum_news_sensitivity","value":0.58}
{"type":"submit_params","pa

{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step","step":33}
{"type":"get_step","step":34}
{"type":"get_step","step":35}
{"type":"get_step","step":36}
{"type":"get_step","step":37}
{"type":"get_step","step":38}
{"type":"get_step","step":39}
{"type":"get_step","step":40}
{"type":"get_step","step":41}
{"type":"get_step","step":42}
{"type":"get_step","step":43}
{"type":"get_step","step":44}
{"type":"get_step","step":45}
{"type":"get_step","step":46}
{"type":"get_step","step":47}
{"type":"g

{"type":"get_step","step":102}
{"type":"get_step","step":103}
{"type":"get_step","step":104}
{"type":"get_step","step":105}
{"type":"get_step","step":106}
{"type":"get_step","step":107}
{"type":"get_step","step":108}
{"type":"get_step","step":109}
{"type":"get_step","step":110}
{"type":"get_step","step":111}
{"type":"get_step","step":112}
{"type":"get_step","step":113}
{"type":"get_step","step":114}
{"type":"get_step","step":115}
{"type":"get_step","step":116}
{"type":"get_step","step":117}
{"type":"get_step","step":118}
{"type":"get_step","step":119}
{"type":"get_step","step":120}
{"type":"get_step","step":121}
{"type":"get_step","step":122}
{"type":"get_step","step":123}
{"type":"get_step","step":124}
{"type":"get_step","step":125}
{"type":"get_step","step":126}
{"type":"get_step","step":127}
{"type":"get_step","step":128}
{"type":"get_step","step":129}
{"type":"get_step","step":130}
{"type":"get_step","step":131}
{"type":"get_step","step":132}
{"type":"get_step","step":133}
{"type":

{"type":"get_step","step":367}
{"type":"get_step","step":368}
{"type":"get_step","step":369}
{"type":"get_step","step":370}
{"type":"get_step","step":371}
{"type":"get_step","step":372}
{"type":"get_step","step":373}
{"type":"get_step","step":374}
{"type":"get_step","step":375}
{"type":"get_step","step":376}
{"type":"get_step","step":377}
{"type":"get_step","step":378}
{"type":"get_step","step":379}
{"type":"get_step","step":380}
{"type":"get_step","step":381}
{"type":"get_step","step":382}
{"type":"get_step","step":383}
{"type":"get_step","step":384}
{"type":"get_step","step":385}
{"type":"get_step","step":386}
{"type":"get_step","step":387}
{"type":"get_step","step":388}
{"type":"get_step","step":389}
{"type":"get_step","step":390}
{"type":"get_step","step":391}
{"type":"get_step","step":392}
{"type":"get_step","step":393}
{"type":"get_step","step":394}
{"type":"get_step","step":395}
{"type":"get_step","step":396}
{"type":"get_step","step":397}
{"type":"get_step","step":398}
{"type":

{"type":"get_step","step":632}
{"type":"get_step","step":633}
{"type":"get_step","step":634}
{"type":"get_step","step":635}
{"type":"get_step","step":636}
{"type":"get_step","step":637}
{"type":"get_step","step":638}
{"type":"get_step","step":639}
{"type":"get_step","step":640}
{"type":"get_step","step":641}
{"type":"get_step","step":642}
{"type":"get_step","step":643}
{"type":"get_step","step":644}
{"type":"get_step","step":645}
{"type":"get_step","step":646}
{"type":"get_step","step":647}
{"type":"get_step","step":648}
{"type":"get_step","step":649}
{"type":"get_step","step":650}
{"type":"get_step","step":651}
{"type":"get_step","step":652}
{"type":"get_step","step":653}
{"type":"get_step","step":654}
{"type":"get_step","step":655}
{"type":"get_step","step":656}
{"type":"get_step","step":657}
{"type":"get_step","step":658}
{"type":"get_step","step":659}
{"type":"get_step","step":660}
{"type":"get_step","step":661}
{"type":"get_step","step":662}
{"type":"get_step","step":663}
{"type":

{"type":"get_step","step":897}
{"type":"get_step","step":898}
{"type":"get_step","step":899}
{"type":"get_step","step":900}
{"type":"get_step","step":901}
{"type":"get_step","step":902}
{"type":"get_step","step":903}
{"type":"get_step","step":904}
{"type":"get_step","step":905}
{"type":"get_step","step":906}
{"type":"get_step","step":907}
{"type":"get_step","step":908}
{"type":"get_step","step":909}
{"type":"get_step","step":910}
{"type":"get_step","step":911}
{"type":"get_step","step":912}
{"type":"get_step","step":913}
{"type":"get_step","step":914}
{"type":"get_step","step":915}
{"type":"get_step","step":916}
{"type":"get_step","step":917}
{"type":"get_step","step":918}
{"type":"get_step","step":919}
{"type":"get_step","step":920}
{"type":"get_step","step":921}
{"type":"get_step","step":922}
{"type":"get_step","step":923}
{"type":"get_step","step":924}
{"type":"get_step","step":925}
{"type":"get_step","step":926}
{"type":"get_step","step":927}
{"type":"get_step","step":928}
{"type":

{"type":"get_step","step":1157}
{"type":"get_step","step":1158}
{"type":"get_step","step":1159}
{"type":"get_step","step":1160}
{"type":"get_step","step":1161}
{"type":"get_step","step":1162}
{"type":"get_step","step":1163}
{"type":"get_step","step":1164}
{"type":"get_step","step":1165}
{"type":"get_step","step":1166}
{"type":"get_step","step":1167}
{"type":"get_step","step":1168}
{"type":"get_step","step":1169}
{"type":"get_step","step":1170}
{"type":"get_step","step":1171}
{"type":"get_step","step":1172}
{"type":"get_step","step":1173}
{"type":"get_step","step":1174}
{"type":"get_step","step":1175}
{"type":"get_step","step":1176}
{"type":"get_step","step":1177}
{"type":"get_step","step":1178}
{"type":"get_step","step":1179}
{"type":"get_step","step":1180}
{"type":"get_step","step":1181}
{"type":"get_step","step":1182}
{"type":"get_step","step":1183}
{"type":"get_step","step":1184}
{"type":"get_step","step":1185}
{"type":"get_step","step":1186}
{"type":"get_step","step":1187}
{"type":

NameError: name 'vix' is not defined

In [ ]:
'''
dj = pd.read_csv('DJIA (Monthly).csv',index_col='DATE', parse_dates=True)
dj = dj[:-1] #Removing last empty row #119 observations
#For some wierd reason know to the excel gods and the Fred people,this has been stored as an object instead of float
#Converting
dj['DJIA'] = dj['DJIA'].apply(pd.to_numeric)
vix = pd.read_csv('vix (Monthly).csv',index_col='DATE', parse_dates=True)
vix = vix[:-1] #Removing last empty row # 120 observations
vix['change']=vix.pct_change(1)
dj['change']= dj.pct_change(1)
vix=vix[1:] #Removing first row #Both have 119 observations now
'''